In [1]:
!pip install git+https://github.com/Farama-Foundation/MAgent2

  Cloning https://github.com/Farama-Foundation/MAgent2 to /tmp/pip-req-build-pisufm4l
  Running command git clone --filter=blob:none --quiet https://github.com/Farama-Foundation/MAgent2 /tmp/pip-req-build-pisufm4l
  Resolved https://github.com/Farama-Foundation/MAgent2 to commit b2ddd49445368cf85d4d4e1edcddae2e28aa1406
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 847.8/847.8 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 60.0 MB/s eta 0:00:00
  Created wheel for magent2: filename=magent2-0.3.3-cp310-cp310-linux_x86_64.whl size=1696076 sha256=deb0f703ce6bef0909674d24805738625c9350687e3899c1c3431730c5266460
  Stored in directory: /tmp/pip-ephem-wheel-cache-yhekp7pd/wheels/e4/8e/bf/51a30bc4038546e23b81c9fb513fe6a8fd916e5a9c5f4291d5
Successfully built magent2


In [2]:
import torch
import torch.nn as nn

class QNetwork(nn.Module):
    def __init__(self, observation_shape, action_shape):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(observation_shape[-1], observation_shape[-1], 3),
            nn.ReLU(),
            nn.Conv2d(observation_shape[-1], observation_shape[-1], 3),
            nn.ReLU(),
        )
        dummy_input = torch.randn(observation_shape).permute(2, 0, 1)
        dummy_output = self.cnn(dummy_input)
        flatten_dim = dummy_output.view(-1).shape[0]
        self.network = nn.Sequential(
            nn.Linear(flatten_dim, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, action_shape),
        )

    def forward(self, x):
        assert len(x.shape) >= 3, "only support magent input observation"
        x = self.cnn(x)
        if len(x.shape) == 3:
            batchsize = 1
        else:
            batchsize = x.shape[0]
        x = x.reshape(batchsize, -1)
        return self.network(x)


In [9]:
from collections import deque, namedtuple
import math
import random
import matplotlib
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

from magent2.environments import battle_v4
# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

def plot_metrics(episode_rewards, episode_losses, show_result=False):
    plt.figure(1)
    plt.clf()
    if show_result:
        plt.title('Result')
    else:
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Value')

    rewards_t = torch.tensor(episode_rewards, dtype=torch.float)
    losses_t = torch.tensor(episode_losses, dtype=torch.float)

    plt.plot(rewards_t.numpy(), label='Reward')
    plt.plot(losses_t.numpy(), label='Loss')

    if len(rewards_t) >= 5:
        rewards_means = rewards_t.unfold(0, 5, 1).mean(1).view(-1)
        rewards_means = torch.cat((torch.zeros(4), rewards_means))
        plt.plot(rewards_means.numpy(), label='Reward (mean)')

    if len(losses_t) >= 5:
        losses_means = losses_t.unfold(0, 5, 1).mean(1).view(-1)
        losses_means = torch.cat((torch.zeros(4), losses_means))
        plt.plot(losses_means.numpy(), label='Loss (mean)')

    plt.legend()
    plt.pause(0.001)
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())


Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

env = battle_v4.env(map_size=45, minimap_mode=False, step_reward=0.01,
                        dead_penalty=-2, attack_penalty=-0.1, attack_opponent_reward=2,
                        max_cycles=200, extra_features=False, render_mode="rgb_array")

BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 1
EPS_END = 0.1
EPS_DECAY = 50
TAU = 0.005
LR = 1e-4

def linear_epsilon(steps_done):
    return max(EPS_END, EPS_START - (EPS_START - EPS_END) * (steps_done / EPS_DECAY))

def policy(observation, q_network):
    global steps_done
    sample = random.random()
    if sample < linear_epsilon(steps_done):
        return env.action_space("red_0").sample()
    else:
        observation = (
            torch.Tensor(observation).float().permute([2, 0, 1]).unsqueeze(0).to(device)
        )
        with torch.no_grad():
            q_values = q_network(observation)
        return torch.argmax(q_values, dim=1).cpu().numpy()[0]

def optimize_model():
    if len(memory) < BATCH_SIZE:
        return

    global running_loss

    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))

    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    state_action_values = policy_net(state_batch).gather(1, action_batch)

    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

    running_loss += loss.item()

    return loss.item()


def save_model(i_episode, policy_net, target_net, optimizer, episode_rewards, episode_losses, path):
    torch.save({
        'episode': i_episode,
        'policy_net_state_dict': policy_net.state_dict(),
        'target_net_state_dict': target_net.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'episode_rewards': episode_rewards,
        'episode_losses': episode_losses,
    }, path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

observation_shape = env.observation_space("blue_0").shape
action_shape = env.action_space("blue_0").n

# Initialize networks
policy_net = QNetwork(observation_shape, action_shape).to(device)
red_policy_net = QNetwork(observation_shape, action_shape).to(device) # for self-play

target_net = QNetwork(observation_shape, action_shape).to(device)
target_net.load_state_dict(policy_net.state_dict())
optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)

# Load pretrained
pretrained_net = QNetwork(observation_shape, action_shape).to(device)
pretrained_net.load_state_dict(torch.load("red.pt", map_location=device, weights_only=True))

try:
    checkpoint = torch.load("models/blue.pt", map_location=device, weights_only=True)
    policy_net.load_state_dict(checkpoint["policy_net_state_dict"])
    target_net.load_state_dict(checkpoint["target_net_state_dict"])
    red_policy_net.load_state_dict(checkpoint["policy_net_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    episode = checkpoint["episode"]
    print(f"Start with episode: {episode}")
except Exception as e:
    print(f"No model found!")
    episode = 0

memory = ReplayMemory(10000)

steps_done = episode
episode_rewards = []
episode_losses = []
running_loss = 0.0
num_episodes = 60

for i_episode in range(episode, num_episodes):
    # Initialize the environment and get its state
    env.reset()
    episode_reward = 0
    running_loss = 0.0
    steps_done += 1

    for agent in env.agent_iter():

        observation, reward, termination, truncation, info = env.last()
        done = termination or truncation

        if done:
            action = None  # Agent is dead
            env.step(action)
        else:
            agent_handle = agent.split("_")[0]
            if agent_handle == "blue":
                action = policy(observation, policy_net)
                env.step(action)

                next_observation, reward, termination, truncation, info = env.last()

                # Convert to tensor before feed into replay buffer
                observation = (torch.Tensor(observation).float().permute([2, 0, 1]).unsqueeze(0).to(device))
                next_observation = (torch.Tensor(next_observation).float().permute([2, 0, 1]).unsqueeze(0).to(device))
                action = torch.tensor([action], device=device, dtype=torch.int64).unsqueeze(0)
                reward = torch.tensor([reward], device=device, dtype=torch.float32)

                # Store the transition in memory
                memory.push(observation, action, next_observation, reward)

                # Perform one step of the optimization (on the policy network)
                optimize_model()

                # Soft update of the target network's weights
                # θ′ ← τ θ + (1 −τ )θ′
                target_net_state_dict = target_net.state_dict()
                policy_net_state_dict = policy_net.state_dict()
                for key in policy_net_state_dict:
                    target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
                target_net.load_state_dict(target_net_state_dict)
                # Update episode reward
                episode_reward += reward
            else:
                # red agent
                action = policy(observation, red_policy_net)
                env.step(action)

        # Periodically update the red agent's policy with the blue agent's learned policy
        if i_episode % 4 == 0 and i_episode < 24:
            # Copy all weights and biases from the blue agent's policy network to the red agent's
            red_policy_net.load_state_dict(policy_net.state_dict())
        elif i_episode == 24: # more complex (pretrained) opponent
            red_policy_net.load_state_dict(pretrained_net.state_dict())



    # Add these lines at the end of each episode
    episode_rewards.append(episode_reward)
    episode_losses.append(running_loss)

    print(f'Episode {i_episode + 1}/{num_episodes}')
    print(f'Total Reward: {episode_reward.item():.2f}')
    print(f'Average Loss: {running_loss:.4f}')
    print(f'Epsilon: {linear_epsilon(steps_done)}')
    print('-' * 40)
    save_model(i_episode, policy_net, target_net, optimizer, episode_rewards, episode_losses, path=f"models/blue_{i_episode}.pt")

plot_metrics(episode_rewards, episode_losses, show_result=True)
plt.ioff()
plt.show()

cuda
Start with episode: 39
Episode 40/100
Total Reward: -48.00
Average Loss: 294.7369
Epsilon: 0.2799999999999999
----------------------------------------
Episode 41/100
Total Reward: -145.00
Average Loss: 57.1847
Epsilon: 0.262
----------------------------------------
Episode 42/100
Total Reward: -54.32
Average Loss: 104.1933
Epsilon: 0.244
----------------------------------------
Episode 43/100
Total Reward: 229.60
Average Loss: 409.3592
Epsilon: 0.22599999999999998
----------------------------------------
Episode 44/100
Total Reward: -118.29
Average Loss: 154.3790
Epsilon: 0.20799999999999996
----------------------------------------
Episode 45/100
Total Reward: -34.01
Average Loss: 56.1005
Epsilon: 0.18999999999999995
----------------------------------------
Episode 46/100
Total Reward: 107.65
Average Loss: 334.6615
Epsilon: 0.17199999999999993
----------------------------------------
Episode 47/100
Total Reward: -113.35
Average Loss: 157.3759
Epsilon: 0.15400000000000003
---------

KeyboardInterrupt: 